In [1]:
import parser_utils
import data_utils

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

tf.keras.backend.set_floatx("float32")

ROOT = "/home/ec2-user/biods220/project/assign2" 

target = ['dextrose', 'acetominophen', 'potassium_y', 'insulin', 'heparin', 'statin',
'heart rate', 'glucose', 'hemocrit', 'WBCs', 'temperature (F)', 'creatinine']

In [3]:
data_utils.build_seq_dataset(ROOT, target)
(
    train_x,
    val_x,
    train_y,
    val_y,
    no_feature_cols,
    no_label_cols,
    test_x,
    test_y,
    x_boolmat_test,
    y_boolmat_test,
    x_boolmat_val,
    y_boolmat_val,
    features,
    labels,
) = data_utils.load_seq_dataset(ROOT, target)

# convert all float64 to float32
train_x = train_x.astype(np.float32)
val_x = val_x.astype(np.float32)
test_x = test_x.astype(np.float32)
train_y = train_y.astype(np.float32)
val_y = val_y.astype(np.float32)
test_y = test_y.astype(np.float32)

print("train shapes ", train_x.shape, train_y.shape)
print("val shapes   ", val_x.shape, val_y.shape)
print("test shapes  ", test_x.shape, test_y.shape)
print("# features   ", len(features))

Building sequence dataset for ['dextrose', 'acetominophen', 'potassium_y', 'insulin', 'heparin', 'statin', 'heart rate', 'glucose', 'hemocrit', 'WBCs', 'temperature (F)', 'creatinine'], saving to /home/ec2-user/biods220/project/assign2/saved_data/['dextrose', 'acetominophen', 'potassium_y', 'insulin', 'heparin', 'statin', 'heart rate', 'glucose', 'hemocrit', 'WBCs', 'temperature (F)', 'creatinine']_15.pkl


/home/ec2-user/biods220/project/assign2/data_utils.py:336: DtypeWarning: Columns (151,153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE)


         BUN   HDL  INR  Inspired O2 Fraction   LDL  PEEP Set    PT   PTT  \
0       22.5  41.0  1.3                  50.0  81.0       5.0  14.0  32.4   
1       22.5  41.0  1.3                  50.0  81.0       5.0  14.0  32.4   
2       22.5  41.0  1.3                  50.0  81.0       5.0  14.0  32.4   
3       22.5  41.0  1.3                  50.0  81.0       5.0  14.0  32.4   
4       22.5  41.0  1.3                  50.0  81.0       5.0  14.0  32.4   
...      ...   ...  ...                   ...   ...       ...   ...   ...   
589405   0.0   0.0  0.0                   0.0   0.0       0.0   0.0   0.0   
589406   0.0   0.0  0.0                   0.0   0.0       0.0   0.0   0.0   
589407   0.0   0.0  0.0                   0.0   0.0       0.0   0.0   0.0   
589408   0.0   0.0  0.0                   0.0   0.0       0.0   0.0   0.0   
589409   0.0   0.0  0.0                   0.0   0.0       0.0   0.0   0.0   

        RBCs  WBCs  ...  tacrolimus  trazodone  vancomycin  vasopressin  \


/home/ec2-user/biods220/project/assign2/data_utils.py:120: RuntimeWarning: invalid value encountered in true_divide
  return (matrix - means) / stds


train shapes  (34030, 15, 229) (34030, 15, 12)
Done saving
train shapes  (34030, 15, 229) (34030, 15, 12)
val shapes    (4862, 15, 229) (4862, 15, 12)
test shapes   (9723, 15, 229) (9723, 15, 12)
# features    229


In [ ]:
labels

In [ ]:
num_stays = 10
time_indices = [time_step for time_step in range(train_x.shape[1])]

label = 'dextrose'

fig, ax = plt.subplots(nrows=num_stays, ncols=1, figsize = (10, 10))
for hospital_stay_idx in range(num_stays):
    ax[hospital_stay_idx].plot(time_indices, train_x[hospital_stay_idx,:,features.index(label)])    
    ax[hospital_stay_idx].plot(time_indices, train_y[hospital_stay_idx,:,labels.index(label)], '--')
fig.show()

In [ ]:
def build_masked_lstm_model(num_timesteps, num_features, lstm_hidden_units=256):
    """
    Return a simple Keras model with a masking single LSTM layer, dropout later,
    and then dense prediction layer.

    Args:
    num_timesteps (int): num timesteps per input data object.
    num_features (int): num features per input data object.
    lstm_hidden_units (int): units in the LSTM layer

    Returns:
    model_lstm (tf.keras.Model) LSTM keras model with output dimension (None,1)
    """
    model_lstm = tf.keras.Sequential()
    model_lstm.add(tf.keras.layers.Masking(mask_value=0, input_shape=(num_timesteps, num_features)))
    model_lstm.add(tf.keras.layers.LSTM(lstm_hidden_units, return_sequences = True))
    model_lstm.add(tf.keras.layers.Dropout(0.5))
    model_lstm.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(no_label_cols)))
    
    for layer in model_lstm.layers[1:]:
        layer.supports_masking = True
    
    return model_lstm

# Code to test the shape is correc #
num_timesteps, num_features = train_x.shape[-2:]
lstm_hidden_units = 256

model = build_masked_lstm_model(num_timesteps, num_features, lstm_hidden_units)
bs = 8
x_batch = train_x[:bs]
print(model(x_batch).shape)  # expect shape (8, 15, 1) # batch size 8, 15 timesteps

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse', 
)

epochs = 10
batch_size = 16
history = model.fit(
    x = train_x,
    y = train_y,
    epochs = epochs, 
    validation_data = (val_x, val_y),
    callbacks = [tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)]
)

In [ ]:
model.save('pretrained.h5')